In [ ]:
# Chapter 18 Notes: Multi-class Classification

# Includes KNN, Dtrees
# Multiclass LDA

# One vs Rest: OvR
# Logisitc Rgression with multiple classes will run OvR
# roc_auc is a meausure


Our dataset consists of clinical data from patients who entered the hospital complaining of chest pain ("angina") during exercise.  The information collected includes:

* `age` : Age of the patient

* `sex` : Sex of the patient

* `cp` : Chest Pain type

    + Value 0: asymptomatic
    + Value 1: typical angina
    + Value 2: atypical angina
    + Value 3: non-anginal pain
   
    
* `trtbps` : resting blood pressure (in mm Hg)

* `chol` : cholesterol in mg/dl fetched via BMI sensor

* `restecg` : resting electrocardiographic results

    + Value 0: normal
    + Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)
    + Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteria

* `thalach` : maximum heart rate achieved during exercise

* `output` : the doctor's diagnosis of whether the patient is at risk for a heart attack
    + 0 = not at risk of heart attack
    + 1 = at risk of heart attack

In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import SVC

In [2]:
ha = pd.read_csv("https://www.dropbox.com/s/aohbr6yb9ifmc8w/heart_attack.csv?dl=1")
ha.head()

,age,sex,cp,trtbps,chol,restecg,thalach,output
0,63,1,3,145,233,0,150,1
1,37,1,2,130,250,1,187,1
2,56,1,1,120,236,1,178,1
3,57,0,0,120,354,1,163,1
4,57,1,0,140,192,1,148,1


In [3]:
ha["cp_is_3"] = np.where(ha["cp"] == 3, 1, 0)
ha["cp_is_2"] = np.where(ha["cp"] == 2, 1, 0)
ha["cp_is_1"] = np.where(ha["cp"] == 1, 1, 0)
ha.head()

,age,sex,cp,trtbps,chol,restecg,thalach,output,cp_is_3,cp_is_2,cp_is_1
0,63,1,3,145,233,0,150,1,1,0,0
1,37,1,2,130,250,1,187,1,0,1,0
2,56,1,1,120,236,1,178,1,0,0,1
3,57,0,0,120,354,1,163,1,0,0,0
4,57,1,0,140,192,1,148,1,0,0,0


## Q2:  OvR

Create a new column in the `ha` dataset called `cp_is_3`, which is equal to `1` if the `cp` variable is equal to `3` and `0` otherwise.

Then, fit a Logistic Regression to predict this new target, and report the **F1 Score**.

Repeat for the other three `cp` categories.  Which category was the OvR approach best at distinguishing?

In [4]:
X3 = ha[["age", "sex", "chol", "trtbps", "restecg", "thalach"]]
y3 = ha["cp_is_3"]
X_train, X_test, y_train, y_test = train_test_split(X3, y3)

In [5]:
logr_pipeline = Pipeline(
  [("logr",  LogisticRegression())]
).set_output(transform = "pandas")

In [6]:
lr_fit3 = logr_pipeline.fit(X_train, y_train)
lr_f13 = cross_val_score(logr_pipeline, X_train, y_train, cv = 5, scoring = "f1").mean()
lr_f13

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.0

In [7]:
y2 = ha["cp_is_2"]
X_train2, X_test2, y_train2, y_test2 = train_test_split(X3, y2)
lr_fit2 = logr_pipeline.fit(X_train2, y_train2)
lr_f12 = cross_val_score(logr_pipeline, X_train2, y_train2, cv = 5, scoring = "f1").mean()
lr_f12

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

0.19047619047619047

In [8]:
y1 = ha["cp_is_1"]
X_train1, X_test1, y_train1, y_test1 = train_test_split(X3, y1)
lr_fit1 = logr_pipeline.fit(X_train1, y_train1)
lr_f11 = cross_val_score(logr_pipeline, X_train1, y_train1, cv = 5, scoring = "f1").mean()
lr_f11

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

0.0

In [9]:
ha["cp_is_0"] = np.where(ha["cp"] == 0, 1, 0)

y0 = ha["cp_is_0"]
X_train0, X_test0, y_train0, y_test0 = train_test_split(X3, y0)
lr_fit0 = logr_pipeline.fit(X_train0, y_train0)
lr_f10 = cross_val_score(logr_pipeline, X_train0, y_train0, cv = 5, scoring = "f1").mean()
lr_f10

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.6233279276622931

In [10]:
lr_f13 + lr_f12 + lr_f11 + lr_f10

0.8138041181384835

OvR was best at distinguishing cp_is_0.

The 4 f1 scores do not add up to 1, because f1 scores are not probabilities, they are model performance metrics.



## Q1: Natural Multiclass Models

Fit a multiclass KNN, Decision Tree, and LDA for the heart disease data; this time predicting the type of chest pain (categories 0 - 3) that a patient experiences.  For the decision tree, plot the fitted tree, and interpret the first couple splits.


In [11]:
knn_pipeline = Pipeline(
  [("knn",  KNeighborsClassifier())]
).set_output(transform = "pandas")

In [13]:
lr_fit4 = knn_pipeline.fit(X_train, y_train)
lr_f14 = cross_val_score(knn_pipeline, X_train, y_train, cv = 5, scoring = "f1").mean()
lr_f14

0.0

## Q3: OvO

Reduce your dataset to only the `0` and `1` types of chest pain.

Then, fit a Logistic Regression to predict between the two groups, and report the **ROC-AUC**.  

Repeat comparing category `0` to `2` and `3`.  Which pair was the OvO approach best at distinguishing?

In [14]:
# 0-1, 0-2, 0-3, 1-2, 1-3, 2-3
# 0-1
ha_new = ha[(ha["cp"] == 0) | (ha["cp"] == 1)]
ha_new = ha_new.drop(columns = ["cp_is_3", "cp_is_2", "cp_is_1", "cp_is_0"])

X5 = ha_new[["age", "sex", "chol", "trtbps", "restecg", "thalach"]]
y5 = ha_new["cp"]

X_train5, X_test5, y_train5, y_test5 = train_test_split(X5, y5)

lr_fit5 = logr_pipeline.fit(X_train5, y_train5)

y_pred5 = lr_fit5.predict(X_test5)
lr_ra5 = cross_val_score(logr_pipeline, X_test5, y_test5, cv = 5, scoring = "roc_auc_ovo").mean()
lr_ra5

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

0.6238095238095238

In [15]:
#0-2
ha_new2 = ha[(ha["cp"] == 0) | (ha["cp"] == 2)]
ha_new2 = ha_new2.drop(columns = ["cp_is_3", "cp_is_2", "cp_is_1", "cp_is_0"])

X6 = ha_new2[["age", "sex", "chol", "trtbps", "restecg", "thalach"]]
y6 = ha_new2["cp"]

X_train6, X_test6, y_train6, y_test6 = train_test_split(X6, y6)

lr_fit6 = logr_pipeline.fit(X_train6, y_train6)
lr_ra6 = cross_val_score(logr_pipeline, X_test6, y_test6, cv = 5, scoring = "roc_auc_ovo").mean()
lr_ra6

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

0.7166666666666667

In [17]:
# 0-3
ha_new3 = ha[(ha["cp"] == 0) | (ha["cp"] == 3)]
ha_new3 = ha_new3.drop(columns = ["cp_is_3", "cp_is_2", "cp_is_1", "cp_is_0"])

X7 = ha_new3[["age", "sex", "chol", "trtbps", "restecg", "thalach"]]
y7 = ha_new3["cp"]

X_train7, X_test7, y_train7, y_test7 = train_test_split(X7, y7)

lr_fit7 = logr_pipeline.fit(X_train7, y_train7)
lr_ra7 = cross_val_score(logr_pipeline, X_test7, y_test7, cv = 5, scoring = "roc_auc_ovo").mean()
lr_ra7

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.5666666666666667

In [18]:
# 1-2
ha_new4 = ha[(ha["cp"] == 1) | (ha["cp"] == 2)]
ha_new4 = ha_new4.drop(columns = ["cp_is_3", "cp_is_2", "cp_is_1", "cp_is_0"])

X8 = ha_new4[["age", "sex", "chol", "trtbps", "restecg", "thalach"]]
y8 = ha_new4["cp"]

X_train8, X_test8, y_train8, y_test8 = train_test_split(X8, y8)

lr_fit8 = logr_pipeline.fit(X_train8, y_train8)
lr_ra8 = cross_val_score(logr_pipeline, X_test8, y_test8, cv = 5, scoring = "roc_auc_ovo").mean()
lr_ra8

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.725

In [19]:
# 1-3
ha_new5 = ha[(ha["cp"] == 1) | (ha["cp"] == 3)]
ha_new5 = ha_new5.drop(columns = ["cp_is_3", "cp_is_2", "cp_is_1", "cp_is_0"])

X9 = ha_new5[["age", "sex", "chol", "trtbps", "restecg", "thalach"]]
y9 = ha_new5["cp"]

X_train9, X_test9, y_train9, y_test9 = train_test_split(X9, y9)

lr_fit9 = logr_pipeline.fit(X_train9, y_train9)
lr_ra9 = cross_val_score(logr_pipeline, X_test9, y_test9, cv = 5, scoring = "roc_auc_ovo").mean()
lr_ra9

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

0.5

In [20]:
# 2-3
ha_new6 = ha[(ha["cp"] == 2) | (ha["cp"] == 3)]
ha_new6 = ha_new6.drop(columns = ["cp_is_3", "cp_is_2", "cp_is_1", "cp_is_0"])

X10 = ha_new6[["age", "sex", "chol", "trtbps", "restecg", "thalach"]]
y10 = ha_new6["cp"]

X_train9, X_test9, y_train9, y_test9 = train_test_split(X9, y9)

lr_fit9 = logr_pipeline.fit(X_train9, y_train9)
lr_ra9 = cross_val_score(logr_pipeline, X_train9, y_train9, cv = 5, scoring = "roc_auc_ovo").mean()
lr_ra9

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

0.6285714285714286

In [21]:
print(lr_ra5)
print(lr_ra6)
print(lr_ra7)
print(lr_ra8)
print(lr_ra9)

0.6238095238095238
0.7166666666666667
0.5666666666666667
0.725
0.6285714285714286
